In [1]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'NLP_project/dataset'
assert FOLDERNAME is not None, "[!] Enter the foldername."

#Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My\ Drive/$FOLDERNAME/

Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/My Drive/NLP_project/dataset/'
/content


In [2]:
import torch

# Confirm that the GPU is detected

assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Found device: Tesla P100-PCIE-16GB, n_gpu: 1


In [3]:
!pip install transformers -q
!pip install pandarallel -q
!pip install sentencepiece -q
!pip install neptune-client -q


     |████████████████████████████████| 3.4 MB 5.2 MB/s 
     |████████████████████████████████| 61 kB 492 kB/s 
     |████████████████████████████████| 3.3 MB 35.8 MB/s 
     |████████████████████████████████| 596 kB 50.6 MB/s 
     |████████████████████████████████| 895 kB 39.3 MB/s 
     |████████████████████████████████| 1.2 MB 5.1 MB/s 
     |████████████████████████████████| 287 kB 5.1 MB/s 
     |████████████████████████████████| 829 kB 45.9 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 180 kB 51.2 MB/s 
     |████████████████████████████████| 131 kB 53.1 MB/s 
     |████████████████████████████████| 8.4 MB 48.4 MB/s 
     |████████████████████████████████| 79 kB 7.0 MB/s 
     |████████████████████████████████| 138 kB 60.2 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 127 kB 52.4 MB/s 
     |████████████████████████████████| 130 kB 49.4 MB/s 
     |███████████████████

In [7]:
import numpy as np
import pandas as pd
import regex as re
import random
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,average_precision_score, precision_score,precision_recall_curve
from tqdm.notebook import tqdm
from tqdm import trange
import warnings
warnings.filterwarnings('ignore')
import pickle
import nltk
import math
import os
import json
import random
import re
import torch
import torch.nn as nn
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel



In [11]:
reviews_path = '/content/drive/MyDrive/gen_datasets/hos_reviews_1.json'
import json
f = open(reviews_path)
data = json.load(f)

In [12]:
len(data.keys())

5703

In [13]:
keys_ = list(data.keys())
hot_reviews = {}
for i in range(80):
  key_ = keys_[i]
  for a in data[key_]['review_list']:
    if(a['rating']!=None):
      hot_reviews[a['text']] = a['rating']

In [14]:
hot_reviews

{"HECK NO!! Some of the world's lousiest food exist right here in POP dining. The only thing that's edible is their breakfast platter. Everything else, burger, pizza, pasta, is horrifying. The pizza is the worse pizza on Earth. There's barely any cheese, the pepperoni is hard, and the dough, crust is half baked. You can still see the layer of dough that has not been fully baked. The beverage and condiment stations are always a hot mess and often has flying insects hovering around certain sections of each station. \n\nThe shopping, everything is overpriced. $10 for a small keychain. The prices are just crazy\nIt's to be expected but even with your Chase Disney debit card, the 10% discount only applies when you spend $50 or more. \n\nVerdict - Shop at a Disney store outlet. As for food, please avoid if you can. Disney dining is the most overrated crap. Don't listen to those YouTube Disney vloggers. Disney dining is garbage but probably fine cuisine for Southerners and hicks from the burb

In [ ]:
reviews_df = pd.read_csv('/content/drive/MyDrive/NLP_project/dataset/yelp_preprocessed_1M.csv')

In [ ]:
print(reviews_df.columns)

Index(['Unnamed: 0', 'date', 'useful', 'text', 'cool', 'funny', 'review_id',
       'user_id', 'stars', 'business_id', 'sentiment'],
      dtype='object')


In [ ]:
reviews_df = reviews_df[0:100]

In [ ]:
model_path = '/content/drive/MyDrive/NLP_project/arg_miningrobertalarge_SEED_4_dense_layer_epoc_3_lr_1e-05_b_s_16_accumulation_steps_2_input_type_kp_arg_topic_preee.pt'

In [ ]:
class NonPoolerTransformer(torch.nn.Module):

    def __init__(self):
        super(NonPoolerTransformer, self).__init__()
        
        #Instantiating Pre trained model object 
        self.model_layer = AutoModel.from_pretrained(model_path)

        #Layers
        # the first dense layer will have 768 if base model is used and 
        # 1024 if large model is used

        self.dense_layer_1 = nn.Linear(1024, 256)
        self.dropout = nn.Dropout(0.4)
        self.dense_layer_2 = nn.Linear(256, 128)
        self.dropout_2 = nn.Dropout(0.2)
        self.cls_layer = nn.Linear(128, 1, bias = True)
        self.sigmoid = nn.Sigmoid()

    def forward(self,input_ids, attention_masks):

        hidden_state = self.model_layer(input_ids=input_ids, attention_mask=attention_masks)[0]
        pooled_output = hidden_state[:, 0]

        x = self.dense_layer_1(pooled_output)
        x = self.dropout(x)
        x_1 = self.dense_layer_2(x)
        x_2 = self.dropout_2(x_1)

        logits = self.cls_layer(x_2)
        output = self.sigmoid(logits)

        return output

In [ ]:
model = torch.load(model_path)
tokenizer = AutoTokenizer.from_pretrained('roberta-large')


Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [18]:
#make reviews and key_points dataset
key_point_df =df = pd.read_csv('/content/drive/MyDrive/NLP_project/dataset/pegasus_hotel_reviews.csv')


positive_kp_df = key_point_df['Positive']
# print(positive_kp_df.head())
negative_kp_df = key_point_df['Negative']
# print(negative_kp_df.head())

reviews_text_df = reviews_df['text']

reviews_list = reviews_text_df.values.tolist()
kp_list = positive_kp_df.values.tolist()
kp_list.extend(negative_kp_df.values.tolist())

print(reviews_list)
print(kp_list)

["HECK NO!! Some of the world's lousiest food exist right here in POP dining. The only thing that's edible is their breakfast platter. Everything else, burger, pizza, pasta, is horrifying. The pizza is the worse pizza on Earth. There's barely any cheese, the pepperoni is hard, and the dough, crust is half baked. You can still see the layer of dough that has not been fully baked. The beverage and condiment stations are always a hot mess and often has flying insects hovering around certain sections of each station. \n\nThe shopping, everything is overpriced. $10 for a small keychain. The prices are just crazy\nIt's to be expected but even with your Chase Disney debit card, the 10% discount only applies when you spend $50 or more. \n\nVerdict - Shop at a Disney store outlet. As for food, please avoid if you can. Disney dining is the most overrated crap. Don't listen to those YouTube Disney vloggers. Disney dining is garbage but probably fine cuisine for Southerners and hicks from the burb

In [20]:
final_list = []

for review in reviews_list:
  for kp in kp_list:
    final_list.append([review,kp])

dataset = pd.DataFrame(final_list,columns = ['review','key_point'])

In [21]:
print(dataset.head())

                                              review                                          key_point
0  HECK NO!! Some of the world's lousiest food ex...  I was so pissed. I was in line for 1 hour and ...
1  HECK NO!! Some of the world's lousiest food ex...                           The food is always great
2  HECK NO!! Some of the world's lousiest food ex...                         Great car. Great service. 
3  HECK NO!! Some of the world's lousiest food ex...   the pool view was great. I would definitely come
4  HECK NO!! Some of the world's lousiest food ex...          Very nice kitchen,dining room,living room


In [22]:
from torch.utils.data import (DataLoader, RandomSampler, WeightedRandomSampler, SequentialSampler, TensorDataset)

In [23]:
def matching_score(dataframe):
  all_attention_masks=[]
  all_input_ids=[]
  match_scores = []

  #print(dataframe.shape[0])
  for i in range(dataframe.shape[0]):
    encoded_input = tokenizer(dataframe.iloc[i]['all_text'], padding='max_length')
    #print(encoded_input['attention_mask'])
    # print(len(encoded_input['attention_mask']))
    all_attention_masks.append(encoded_input['attention_mask'][0:512])
    all_input_ids.append(encoded_input['input_ids'][0:512])

  all_attention_masks = torch.tensor(all_attention_masks).squeeze()
  all_input_ids = torch.tensor(all_input_ids).squeeze()


  dataset = TensorDataset(all_input_ids, all_attention_masks)


  test_dataloader = DataLoader(dataset, batch_size=1)
      
  with torch.no_grad():
      acc_epoch = []

      epoch_iterator = tqdm(test_dataloader, desc="Iteration")
      for step, batch in enumerate(epoch_iterator):
          model.eval()
              

          b_input_ids, b_input_mask = batch[0].to(device), batch[1].to(device)
          ypred = model(b_input_ids, b_input_mask)
          # print(ypred)
          match_scores.append(ypred[0][0].item())
  return match_scores


In [24]:
sentence_df = pd.DataFrame([], columns =['Review', 'Sentence', 'Key_point'])

In [25]:
for index, row in dataset.iterrows():
  if row.review!= None:
    splitted = row.review.split('.')
    for sentence in splitted:
      print(sentence)
      new_row = [row.review, sentence, row.key_point]
      sentence_df.loc[len(sentence_df.index)] = new_row
  else:
      continue

Streaming output truncated to the last 5000 lines.
The service was good
 It was nice to meet the restaurant manager Jon
 He stopped by and chatted with us
 I really enjoy that kind of service! It shows they care!
I met my friend for lunch here yesterday
 A great spot to meet friends for a really great lunch


We started out with the calamari and steak flat bread
 They were both awesome
 The calamari were like fat stubby french fry size, perfectly cooked, not rubbery with a crunchy coating and a delicious marianara on the side
 The flat bread was super crispy and it had great flavor! It had delicious marinated thinly sliced steak with some micro greens tossed in some kind of dressing and cheese too
 

Next we had the tomato soup
 It was so good
 Not completely pureed but it wasn't chunky either
 It was topped with a goat cheese dollop and finely minced green onions
 I don't normally like goat cheese but with the tomato soup it was perfect!

For the final course we split a field greens s

In [26]:
sentence_df.head()

,Review,Sentence,Key_point
0,HECK NO!! Some of the world's lousiest food ex...,HECK NO!! Some of the world's lousiest food ex...,I was so pissed. I was in line for 1 hour and ...
1,HECK NO!! Some of the world's lousiest food ex...,The only thing that's edible is their breakfa...,I was so pissed. I was in line for 1 hour and ...
2,HECK NO!! Some of the world's lousiest food ex...,"Everything else, burger, pizza, pasta, is hor...",I was so pissed. I was in line for 1 hour and ...
3,HECK NO!! Some of the world's lousiest food ex...,The pizza is the worse pizza on Earth,I was so pissed. I was in line for 1 hour and ...
4,HECK NO!! Some of the world's lousiest food ex...,"There's barely any cheese, the pepperoni is h...",I was so pissed. I was in line for 1 hour and ...


In [27]:
sentence_df['all_text'] = sentence_df['Sentence']+sentence_df['Key_point']

In [28]:
sentence_df.head()

,Review,Sentence,Key_point,all_text
0,HECK NO!! Some of the world's lousiest food ex...,HECK NO!! Some of the world's lousiest food ex...,I was so pissed. I was in line for 1 hour and ...,HECK NO!! Some of the world's lousiest food ex...
1,HECK NO!! Some of the world's lousiest food ex...,The only thing that's edible is their breakfa...,I was so pissed. I was in line for 1 hour and ...,The only thing that's edible is their breakfa...
2,HECK NO!! Some of the world's lousiest food ex...,"Everything else, burger, pizza, pasta, is hor...",I was so pissed. I was in line for 1 hour and ...,"Everything else, burger, pizza, pasta, is hor..."
3,HECK NO!! Some of the world's lousiest food ex...,The pizza is the worse pizza on Earth,I was so pissed. I was in line for 1 hour and ...,The pizza is the worse pizza on EarthI was so...
4,HECK NO!! Some of the world's lousiest food ex...,"There's barely any cheese, the pepperoni is h...",I was so pissed. I was in line for 1 hour and ...,"There's barely any cheese, the pepperoni is h..."


In [29]:
sentence_df.to_csv('sentence_splitted_review_kp.csv')

In [30]:
print(sentence_df.shape)

(23504, 4)


In [31]:
sentence_df['score'] = matching_score(sentence_df) 

In [32]:
sentence_df.to_csv('sentence_df_scores.csv')

In [34]:
sentence_df.head()

,Review,Sentence,Key_point,all_text,score
0,HECK NO!! Some of the world's lousiest food ex...,HECK NO!! Some of the world's lousiest food ex...,I was so pissed. I was in line for 1 hour and ...,HECK NO!! Some of the world's lousiest food ex...,0.016257
1,HECK NO!! Some of the world's lousiest food ex...,The only thing that's edible is their breakfa...,I was so pissed. I was in line for 1 hour and ...,The only thing that's edible is their breakfa...,0.099707
2,HECK NO!! Some of the world's lousiest food ex...,"Everything else, burger, pizza, pasta, is hor...",I was so pissed. I was in line for 1 hour and ...,"Everything else, burger, pizza, pasta, is hor...",0.065177
3,HECK NO!! Some of the world's lousiest food ex...,The pizza is the worse pizza on Earth,I was so pissed. I was in line for 1 hour and ...,The pizza is the worse pizza on EarthI was so...,0.023128
4,HECK NO!! Some of the world's lousiest food ex...,"There's barely any cheese, the pepperoni is h...",I was so pissed. I was in line for 1 hour and ...,"There's barely any cheese, the pepperoni is h...",0.011633


In [35]:
matched_df = pd.DataFrame([], columns =['Review', 'Key_point', 'Score'])
new_df = sentence_df

In [ ]:
pos_matched_df = pd.DataFrame([], columns =['Review', 'Key_point', 'Score'])
neg_matched_df = pd.DataFrame([], columns =['Review', 'Key_point', 'Score'])
no_matched_df = pd.DataFrame([], columns =['Review', 'Key_point', 'Score'])

i=0
while i<new_df.shape[0]:
  flag = False
  for j in range(60):
    rowSeries = new_df.iloc[i+j]
    if rowSeries['score']>0.95:
      flag = True
      new_row = [rowSeries.Review, rowSeries.Key_point, rowSeries.Score]
      pos_matched_df.loc[len(pos_matched_df.index)] = new_row
  for j in range(60,120):
    rowSeries = new_df.iloc[i+j]
    if rowSeries['score']>0.95:
      flag = True
      new_row = [rowSeries.Review, rowSeries.Key_point, rowSeries.Score]
      neg_matched_df.loc[len(neg_matched_df.index)] = new_row
  if flag == False:
    row = new_df.iloc[i]
    new_row = [row.Review, row.Key_point, 0]
    no_matched_df.loc[len(no_matched_df.index)] = new_row
  i = i+120

In [ ]:
# pos_matched_df.head()

In [ ]:
# neg_matched_df.head()

In [ ]:
# no_matched_df.head()
# no_matched_df['Review'][0]

In [40]:
pos_kp_coverage = ((pos_matched_df.groupby('Key_point').size()/reviews_df.shape[0])*100)
pos_kp_coverage_top10 = pos_kp_coverage.nlargest(10)
print(pos_kp_coverage_top10)

Key_point
                                              Great food. Great atmosphere.  14.807486
                                               Hotel is nice and kept clean  12.406521
                                                  Crazy cheap for good food  12.209001
                                            The room was clean and spacious  11.108252
                      We had a great time and were able to explore the city  10.707411
                                     each visit has been a great experience   9.507481
                         This was one of the best service I have ever seen.   8.707379
                                             Absolutely fabulous experience   8.105066
               Very nice decor. Elegant bedrooms with very comfortable beds   7.854932
 I have stayed at this hotel many times and have never had a bad experience   7.208598
dtype: float64


In [41]:
neg_kp_coverage = ((neg_matched_df.groupby('Key_point').size()/reviews_df.shape[0])*100)
neg_kp_coverage_top10 = neg_kp_coverage.nlargest(10)
print(neg_kp_coverage_top10)

Key_point
                          Expensive, Not Worth it !  15.708621
                 It’s disgusting and way overpriced  14.767626
             Disappointed, the room was very small.  11.203464
             Very poor service and you have to haul  10.503615
                       Customer service is terrible   9.203570
  Mansion it self is a bit pricey for what you get.   8.956838
                       Dissappointed in sales staff   8.405210
                                    the room stinks   8.103985
 Totally inexcusable, the staff doesn’t care at all   7.957228
       Was full of mold, cracks and rust in my room   7.793220
dtype: float64


In [42]:
neg_kp_coverage.to_csv("neg_kp_coverage.csv")

In [ ]:
pos_kp_coverage.to_csv("pos_kp_coverage.csv")